In [ ]:
players_info =[]

In [ ]:
def parse_player_data(soup):
    # Extracting player details
    paras = soup.find_all('p', class_='ds-text-tight-s ds-font-regular ds-uppercase ds-text-typo-mid3')
    batting_style = None
    bowling_style = None
    playing_role = None
    for para in paras:
        if para.text == 'Batting Style':
            batting_style = para.find_next('span').text
        elif para.text == 'Bowling Style':
            bowling_style = para.find_next('span').text
        elif para.text == 'Playing Role':
            playing_role = para.find_next('span').text


    # Extracting description

    description = soup.select_one('div',class_='ci-player-bio-content').text

    return {
        "battingStyle": batting_style,
        "bowlingStyle": bowling_style,
        "playingRole": playing_role,
        "content": description
    }


In [ ]:
def next_stage_stage3(name, team, url):
    # Step 1: Navigate to the match URL
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Proceed with parsing the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Step 2: Parse player details
        final_data = parse_player_data(soup)

        # Step 3: Collect and store final player data
        player_info = {
            "name": name,
            "team": team,
            "battingStyle": final_data['battingStyle'],
            "bowlingStyle": final_data['bowlingStyle'],
            "playingRole": final_data['playingRole'],
            "description": final_data['content'],
        }

        # Here, you would typically store or process the player_info data
        players_info.append(player_info)
    else:
        print("Failed to retrieve data from the website. Status code:", response.status_code)



In [ ]:
def next_stage_stage2(url):
    # Step 1: Navigate to the match URL
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Proceed with parsing the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Step 2: Parse player data
        players_data = []

        # Identifying teams
        match_details = soup.find_all('div',class_='ds-bg-fill-canvas')
        # print(match_details)

        teams= match_details[0].find_all('span',class_='ds-text-tight-xs')
        team1 = teams[0].text.replace(" Innings","")
        team2 = teams[1].text.replace(" Innings","")
        # Extracting batting players
        table_div= soup.find_all('div',class_ = 'ds-p-0')
        first_innings_rows = (table_div[1].find_all('table'))[0].select('tbody > tr')
        second_innings_rows = (table_div[2].find_all('table'))[0].select('tbody > tr')


        for row in first_innings_rows:
            if len(row.find_all("td")) >= 8:
                outerSpan = (row.select('td')[0].select('td span.ds-text-tight-s.ds-font-medium.ds-text-typo.ds-underline.ds-decoration-ui-stroke.hover\\:ds-text-typo-primary.hover\\:ds-decoration-ui-stroke-primary.ds-block'))[0]
                name = outerSpan.find_all('span')[0].text.strip()
                link = "https://www.espncricinfo.com" + row.select('td')[0].find('a')['href']
                players_data.append({"name": name, "team": team1, "link": link})

        for row in second_innings_rows:
            if len(row.find_all("td")) >= 8:
                outerSpan = (row.select('td')[0].select('td span.ds-text-tight-s.ds-font-medium.ds-text-typo.ds-underline.ds-decoration-ui-stroke.hover\\:ds-text-typo-primary.hover\\:ds-decoration-ui-stroke-primary.ds-block'))[0]
                name = outerSpan.find_all('span')[0].text.strip()
                link = "https://www.espncricinfo.com" + row.select('td')[0].find('a')['href']
                players_data.append({"name": name, "team": team2, "link": link})

        # Extracting bowling players

        first_innings_bowlers = (table_div[1].find_all('table'))[1].select('tbody > tr')
        second_innings_bowlers = (table_div[2].find_all('table'))[1].select('tbody > tr')

        for row in first_innings_bowlers:
            if len(row.find_all("td")) >= 11:
                name = (row.select('td')[0].select('span.ds-text-tight-s.ds-font-medium.ds-text-typo.ds-underline.ds-decoration-ui-stroke.hover\\:ds-text-typo-primary.hover\\:ds-decoration-ui-stroke-primary.ds-block'))[0].text.strip()
                link = "https://www.espncricinfo.com" + row.select('td')[0].find('a')['href']
                players_data.append({"name": name, "team": team2, "link": link})

        for row in second_innings_bowlers:
            if len(row.find_all("td")) >= 11:
                name = (row.select('td')[0].select('span.ds-text-tight-s.ds-font-medium.ds-text-typo.ds-underline.ds-decoration-ui-stroke.hover\\:ds-text-typo-primary.hover\\:ds-decoration-ui-stroke-primary.ds-block'))[0].text.strip()
                link = "https://www.espncricinfo.com" + row.select('td')[0].find('a')['href']
                players_data.append({"name": name, "team": team1, "link": link})

        # Moving to the next stage with each player data
        for player in players_data:
            next_stage_stage3(name=player['name'], team=player['team'], url=player['link'])
    else:
        print("Failed to retrieve data from the website. Status code:", response.status_code)


In [ ]:
import requests
from bs4 import BeautifulSoup

# Step 1: Navigate to the webpage
url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Proceed with parsing the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Step 2: Parse match summary links
    match_summary_links = []
    table_body = soup.select('#main-container > div.ds-relative > div > div.ds-flex.ds-space-x-5 > div.ds-grow > div:nth-child(2) > div > div:nth-child(1) > div.ds-overflow-x-auto.ds-scrollbar-hide > table > tbody')[0]
    table_rows = table_body.find_all('tr')

    for row in table_rows:
        tds = row.find_all('td')
        row_url = "https://www.espncricinfo.com" + tds[6].find('a')['href']
        match_summary_links.append(row_url)

    # Moving to the next stage with each match URL
    for link in match_summary_links:
        next_stage_stage2(url=link)
else:
    print("Failed to retrieve data from the website. Status code:", response.status_code)



In [ ]:
import json

# Specify the file path where you want to store the JSON file
file_path = 't20_wc_player_info.json'

# Writing the list to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(players_info, json_file, indent=4)

print(f"List has been stored in {file_path}")


List has been stored in t20_wc_player_info.json
